In [ ]:
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import torch
from cmcrameri import cm

In [ ]:
plt.style.use(Path("../meta/colorblind_friendly.mplstyle"))
matplotlib.rcParams.update({"font.family": "serif", "font.size": 8})


# station locations
stationlist_all = pl.read_csv("../meta/stations_all.csv")
stations_all = torch.tensor(
    np.vstack((stationlist_all["X"].to_numpy(), stationlist_all["Y"].to_numpy())).T
)

# receiver stations
stationlist_rcv = pl.read_csv("../meta/stations_receivers.csv")
names_receivers = stationlist_rcv["station"]
stations_receivers = torch.tensor(
    np.vstack((stationlist_rcv["X"].to_numpy(), stationlist_rcv["Y"].to_numpy())).T
)

# auxiliary stations
stationlist_aux = pl.read_csv("../meta/stations_auxiliary.csv")
stations_auxiliary = torch.tensor(
    np.vstack((stationlist_aux["X"].to_numpy(), stationlist_aux["Y"].to_numpy())).T
)

master_idx = names_receivers.to_numpy().tolist().index("OMV.GDT")

_cm = 1 / 2.54

fig, ax = plt.subplots(figsize=(1.45 * 9 * _cm, 9 * _cm))
ax.scatter(*stations_all.T, s=5, lw=0, c="#CCC")
ax.scatter(*stations_receivers.T, s=5, lw=0)
ax.scatter(*stations_receivers[master_idx].T, s=100, marker="v", lw=0.5, ec="k")
ax.scatter(*stations_auxiliary.T, s=5, lw=0, c="C4")

ax.set(xlim=(-11, 11), ylim=(-8, 7), aspect="equal")

In [ ]:
# load correlations
c1_correlations_filt_both = torch.load(
    "../data/correlations_for_c1_filt_synth_both.pt", weights_only=False
)
c1_correlations_filt_boundary = torch.load(
    "../data/correlations_for_c1_filt_synth_boundary.pt", weights_only=False
)

c2_correlations_unstacked_filt_both = torch.load(
    "../data/c2_correlations_unstacked_filt_synth_both.pt", weights_only=False
)

c2_correlations_unstacked_filt_boundary = torch.load(
    "../data/c2_correlations_unstacked_filt_synth_boundary.pt", weights_only=False
)

# must match compute_correlations.ipynb
sampling_rate = 5
length_of_oneside = 300
window_length = length_of_oneside

times = torch.arange(0, 2 * length_of_oneside + 1 / sampling_rate, 1 / sampling_rate)
lapse_times_c1 = torch.arange(
    -length_of_oneside, length_of_oneside + 1 / sampling_rate, 1 / sampling_rate
)
lapse_times_c2 = torch.arange(-window_length / 2, window_length / 2, 1 / sampling_rate)

In [ ]:
# average causal and anti-causal C2 correlations
c2_correlations_unstacked_filt_both = c2_correlations_unstacked_filt_both.mean(dim=0)
c2_correlations_unstacked_filt_boundary = c2_correlations_unstacked_filt_boundary.mean(
    dim=0
)

# average over all auxiliary stations
c2_correlations_filt_both = c2_correlations_unstacked_filt_both.mean(dim=1)
c2_correlations_filt_boundary = c2_correlations_unstacked_filt_boundary.mean(dim=1)

c2_correlations_filt_both.shape, c2_correlations_filt_boundary.shape

In [ ]:
# potentially useful stuff
idx_closest_to_0 = np.abs(stations_receivers[:, 0]).argmin()
c1_zerolag_idx = lapse_times_c1.abs().argmin()
c2_zerolag_idx = lapse_times_c2.abs().argmin()

c1_focal_spots = [
    c1_correlations_filt_boundary[:, c1_zerolag_idx],
    c1_correlations_filt_both[:, c1_zerolag_idx],
]

c2_focal_spots = [
    c2_correlations_filt_boundary[:, c2_zerolag_idx],
    c2_correlations_filt_both[:, c2_zerolag_idx],
]

# normalise all focal spots to max
for i in range(len(c1_focal_spots)):
    c1_focal_spots[i] /= c1_focal_spots[i].abs().max()
    c2_focal_spots[i] /= c2_focal_spots[i].abs().max()

In [ ]:
# another figure that compares full circle stack against C1 and ideal C1
# rows are: c1 realistic and c1 ideal
# columns are: c3 full circle, c1s, diffs
fig, axs = plt.subplots(3, 3, figsize=(8.3 * _cm, 8.3 * _cm))

for ax, fs, label, description in zip(
    axs.flat,
    [
        c1_focal_spots[0],
        c1_focal_spots[1],
        c1_focal_spots[1] - c1_focal_spots[0],
        c2_focal_spots[0],
        c2_focal_spots[1],
        c2_focal_spots[1] - c2_focal_spots[0],
        c2_focal_spots[0] - c1_focal_spots[0],
        c2_focal_spots[1] - c1_focal_spots[1],
    ],
    ["a", "b", "c", "d", "e", "f", "g", "h"],
    (
        r"$C^{ideal}_1$",
        r"$C_1$",
        "b) – a)",
        r"$C^{ideal}_2$",
        r"$C^{all}_2$",
        "e) – d)",
        "d) – a)",
        "e) – b)",
    ),
):
    sct = ax.scatter(
        *stations_receivers.T,
        c=fs,
        cmap=cm.broc,
        s=2.5,
        vmin=-1,
        vmax=1,
        lw=0,
    )

    ax.set(
        xlim=(-4.5, 4.5),
        ylim=(-4.5, 4.5),
        xticks=(-4, 0, 4),
        yticks=(-4, 0, 4),
        xticklabels=[],
        yticklabels=[],
        aspect="equal",
    )
    ax.scatter(
        *stations_receivers[master_idx].T, marker="v", c="#FFA90E", ec="k", s=25, lw=0.5
    )

    ax.text(
        0.05,
        0.95,
        description,
        ha="left",
        va="top",
        transform=ax.transAxes,
        fontsize=7,
        bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="none", alpha=0.7),
    )
    ax.set_title(f"{label})", loc="left", fontsize=8, pad=2)

for ax in axs[-1]:
    ax.set_xlabel("Distance [km]", labelpad=0, fontsize=7)
    ax.set_xticklabels([-4, 0, 4], fontsize=7)
for ax in axs[:, 0]:
    ax.set_ylabel("Distance [km]", labelpad=0, fontsize=7)
    ax.set_yticklabels([-4, 0, 4], fontsize=7)


axs[2, 2].axis("off")  # turn off the axes

# colorbar
x0, y0, w, h = axs[2, 2].get_position().bounds
cbar_ax = fig.add_axes([x0, y0 + h - 0.02, w, 0.02])
cbar = fig.colorbar(sct, cax=cbar_ax, orientation="horizontal")
cbar.set_label("Simulated\namplitudes", fontsize=7, labelpad=0)
cbar.ax.tick_params(labelsize=7)
# set ticks to -1, 0, 1
cbar.set_ticks([-1, 0, 1])
cbar.set_ticklabels(["-1", "0", "1"])

fig.savefig("../figures/figure5.png", dpi=300, bbox_inches="tight")